In [1]:
from nba_api.stats.static import players
from nba_api.stats.endpoints import commonplayerinfo
from IPython.display import clear_output
import pandas as pd 

In [15]:
def get_player_info(player_ids=None):
    if not player_ids:
        all_players = [p for p in players.get_players()]
        player_list = [p for p in players.get_players()]
    else:
        all_players = [p for p in player_ids]
        player_list = [p for p in player_ids]
    
    attempts = {p['id']:1 for p in all_players}
    player_infos = []
    total = len(all_players)
    while player_list != []:
        for player in all_players:
            clear_output(wait=True)
            if player in player_list:
                name = player['full_name']
                completed = total - len(player_list)
                print(f'{round((completed/total)*100,2)}% completion')
                print(f'Retrieving player info for {name}')
                try:
                    player_info = commonplayerinfo.CommonPlayerInfo(player_id=player['id']).get_data_frames()[0][['PERSON_ID','DISPLAY_FIRST_LAST','FROM_YEAR','TO_YEAR']]
                    player_infos.append(player_info)
                    player_list.remove(player)
                except:
                    i = attempts[player['id']]
                    if i < 50:
                        print(f'Attempt {i} FAILED to retrieve info for {name}. Queued for retry.')
                        attempts[player['id']] = i + 1
                    else:
                        print(f'Attempt {i} FAILED to retrieve info for {name}. Max attempts for player has been met.')
                        player_list.remove(player)
            else:
                continue
                        
    print('100%')
    df = pd.concat(player_infos)
    df = df.reset_index(drop=True)
    df.columns = ['player_id', 'player_name', 'year_start', 'year_end']
    return df

In [3]:
df = get_player_info()
df

100%


,player_id,player_name,year_start,year_end
0,76001,Alaa Abdelnaby,1990,1994
0,76002,Zaid Abdul-Aziz,1968,1977
0,76003,Kareem Abdul-Jabbar,1969,1988
0,51,Mahmoud Abdul-Rauf,1990,2000
0,1505,Tariq Abdul-Wahad,1997,2003
...,...,...,...,...
0,78040,Joe Ruklick,1959,1961
0,1730,Brian Skinner,1998,2011
0,78179,Charles Smith,1989,1995
0,201160,Jason Smith,2007,2018
